In [6]:
import db_manager
import parser

path_target = '/home/kmh/Dataset/articles.A-B.xml'

class Main:
    def __init__(self):
        self.parser = parser.Parser()
        self.db_manager = db_manager.DB_manager()
        
    def run(self):
#         i = 1
        for xml in list(map(lambda s : s.strip(), open(path_target + '.txt', 'r').readlines())) :
#             if i == 10 :
#                 break
#             i = i + 1
            
            try :
                self.parser.set_article(path_target + '/' + xml)
                sql = self.db_manager.sql_insert_into_pmid(self.parser.get_pmid(), self.parser.get_abstract()) \
                        + self.db_manager.sql_insert_into_abstract(self.parser.get_pmid(), self.parser.get_su(), self.parser.get_ppub()) \
                        + self.db_manager.sql_get_all_sentence(self.parser.get_pmid(), self.parser.get_map_label(), self.parser.get_origin_label(), self.parser.get_sentence())
                self.db_manager.commit(sql.encode())
                
#                 self.parser.print()
#                 print(sql)
#                 print('\n\n')
            except Exception as error:
                print(xml)
                print(error)
                continue
                
        self.db_manager.finish()

In [7]:
if __name__ == "__main__" :
    Main().run()

KeyboardInterrupt: 